<a href="https://colab.research.google.com/github/thaismrb/Lista2/blob/main/Lista2_Processamento_Imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install --upgrade pip
!pip install opencv-python-headless matplotlib scikit-image scikit-learn


In [44]:
from google.colab import files

uploaded = files.upload()


In [45]:
import os

os.makedirs('data', exist_ok=True)

for f in uploaded.keys():
    os.rename(f, f"data/{f}")

os.listdir('data')


['imagem3.jpg',
 '.ipynb_checkpoints',
 'imagem2.jpg',
 'imagem1.jpg',
 'imagem_binaria.png']

In [46]:
os.listdir('data')


['imagem3.jpg',
 '.ipynb_checkpoints',
 'imagem2.jpg',
 'imagem1.jpg',
 'imagem_binaria.png']

In [47]:
import os
os.makedirs('src', exist_ok=True)
print("Pasta 'src' criada!")


Pasta 'src' criada!


In [48]:
%%writefile src/otsu.py
import cv2
import matplotlib.pyplot as plt
import os
from pathlib import Path

def run(image_path, out_dir='outputs'):
    os.makedirs(out_dir, exist_ok=True)
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    limiar, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    print(f"Limiar de Otsu encontrado: {limiar}")

    base = Path(image_path).stem
    cv2.imwrite(f"{out_dir}/{base}_otsu_mask.png", mask)
    segmented = cv2.bitwise_and(img, img, mask=mask)
    cv2.imwrite(f"{out_dir}/{base}_otsu_segmented.png", segmented)

    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)); plt.title('Original'); plt.axis('off')
    plt.subplot(1,3,2); plt.hist(gray.ravel(),256,[0,256]); plt.axvline(limiar,color='r'); plt.title('Histograma')
    plt.subplot(1,3,3); plt.imshow(mask, cmap='gray'); plt.title('Otsu - Binária'); plt.axis('off')
    plt.show()

if __name__ == '__main__':
    import sys
    if len(sys.argv) < 2:
        print("Uso: python src/otsu.py data/imagem1.jpg")
    else:
        run(sys.argv[1])


Overwriting src/otsu.py


In [49]:
%%writefile src/kmeans.py
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import os
from pathlib import Path

def run(image_path, K=3, out_dir='outputs'):
    os.makedirs(out_dir, exist_ok=True)
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h,w,_ = img_rgb.shape
    X = img_rgb.reshape((-1,3)).astype('float32')
    kmeans = KMeans(n_clusters=K, n_init=10, random_state=42).fit(X)
    centers = kmeans.cluster_centers_.astype('uint8')
    labels = kmeans.labels_
    seg = centers[labels].reshape((h,w,3))

    base = Path(image_path).stem
    cv2.imwrite(f"{out_dir}/{base}_kmeans_K{K}.png", cv2.cvtColor(seg, cv2.COLOR_RGB2BGR))

    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1); plt.imshow(img_rgb); plt.title('Original'); plt.axis('off')
    plt.subplot(1,2,2); plt.imshow(seg); plt.title(f'K-Means (K={K})'); plt.axis('off')
    plt.show()

if __name__ == '__main__':
    import sys
    if len(sys.argv) < 2:
        print("Uso: python src/kmeans.py data/imagem1.jpg")
    else:
        run(sys.argv[1])


Overwriting src/kmeans.py


In [50]:
%%writefile src/chain_code.py
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

DIRECOES = [(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1),(0,1),(1,1)]

def extrair_chain(mask):
    contornos, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not contornos:
        return None, []
    contorno = max(contornos, key=cv2.contourArea).squeeze()
    if (contorno[0]==contorno[-1]).all():
        contorno = contorno[:-1]
    codigos = []
    for i in range(len(contorno)-1):
        dx = np.sign(contorno[i+1][0] - contorno[i][0])
        dy = np.sign(contorno[i+1][1] - contorno[i][1])
        try:
            codigos.append(DIRECOES.index((dx,dy)))
        except ValueError:
            d = min(range(len(DIRECOES)), key=lambda k: (DIRECOES[k][0]-dx)**2 + (DIRECOES[k][1]-dy)**2)
            codigos.append(d)
    return tuple(contorno[0]), codigos

def run(mask_path, out_dir='outputs'):
    os.makedirs(out_dir, exist_ok=True)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    inicio, codigos = extrair_chain(mask)
    if inicio is None:
        print("Nenhum contorno encontrado.")
        return
    print("Início:", inicio, "Comprimento:", len(codigos))
    pontos = [inicio]
    for c in codigos:
        dx,dy = DIRECOES[c]
        x,y = pontos[-1]
        pontos.append((x+dx, y+dy))
    pontos = np.array(pontos)
    img = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    for p in pontos:
        cv2.circle(img, tuple(int(v) for v in p), 1, (0,0,255), -1)
    base = Path(mask_path).stem
    cv2.imwrite(f"{out_dir}/{base}_chain_overlay.png", img)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Contorno (Chain Code)")
    plt.axis('off')
    plt.show()
    print("Primeiros 20 códigos:", codigos[:20])

if __name__ == '__main__':
    import sys
    if len(sys.argv) < 2:
        print("Uso: python src/chain_code.py outputs/imagem1_otsu_mask.png")
    else:
        run(sys.argv[1])


Overwriting src/chain_code.py


In [51]:
!ls src


chain_code.py  kmeans.py  otsu.py


In [52]:
!python src/otsu.py data/imagem1.jpg


Limiar de Otsu encontrado: 149.0
/content/src/otsu.py:20: MatplotlibDeprecationWarning: Passing the range parameter of hist() positionally is deprecated since Matplotlib 3.9; the parameter will become keyword-only in 3.11.
  plt.subplot(1,3,2); plt.hist(gray.ravel(),256,[0,256]); plt.axvline(limiar,color='r'); plt.title('Histograma')
Figure(1200x400)


In [53]:
!python src/otsu.py data/imagem2.jpg

Limiar de Otsu encontrado: 97.0
/content/src/otsu.py:20: MatplotlibDeprecationWarning: Passing the range parameter of hist() positionally is deprecated since Matplotlib 3.9; the parameter will become keyword-only in 3.11.
  plt.subplot(1,3,2); plt.hist(gray.ravel(),256,[0,256]); plt.axvline(limiar,color='r'); plt.title('Histograma')
Figure(1200x400)


In [54]:
!python src/otsu.py data/imagem3.jpg

Limiar de Otsu encontrado: 114.0
/content/src/otsu.py:20: MatplotlibDeprecationWarning: Passing the range parameter of hist() positionally is deprecated since Matplotlib 3.9; the parameter will become keyword-only in 3.11.
  plt.subplot(1,3,2); plt.hist(gray.ravel(),256,[0,256]); plt.axvline(limiar,color='r'); plt.title('Histograma')
Figure(1200x400)


In [55]:
!python src/kmeans.py data/imagem1.jpg


Figure(1000x500)


In [56]:
!python src/kmeans.py data/imagem2.jpg

Figure(1000x500)


In [57]:
!python src/kmeans.py data/imagem3.jpg

Figure(1000x500)


In [58]:
!python src/chain_code.py outputs/imagem1_otsu_mask.png


Início: (np.int32(0), np.int32(0)) Comprimento: 1953
Figure(640x480)
Primeiros 20 códigos: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]


In [59]:
!python src/chain_code.py outputs/imagem2_otsu_mask.png

Início: (np.int32(667), np.int32(0)) Comprimento: 3087
Figure(640x480)
Primeiros 20 códigos: [7, 6, 7, 6, 7, 7, 6, 7, 6, 7, 7, 6, 7, 7, 6, 7, 7, 7, 7, 6]


In [60]:
!python src/chain_code.py outputs/imagem3_otsu_mask.png

Início: (np.int32(134), np.int32(121)) Comprimento: 2163
Figure(640x480)
Primeiros 20 códigos: [6, 6, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [61]:
!ls outputs


imagem1_kmeans_K3.png		     imagem2_otsu_mask.png
imagem1_otsu_mask_chain_overlay.png  imagem2_otsu_segmented.png
imagem1_otsu_mask.png		     imagem3_kmeans_K3.png
imagem1_otsu_segmented.png	     imagem3_otsu_mask_chain_overlay.png
imagem2_kmeans_K3.png		     imagem3_otsu_mask.png
imagem2_otsu_mask_chain_overlay.png  imagem3_otsu_segmented.png
